# Lab 1
### Jason McDonald, Miguel Bonilla, Zachary Bunn

## AirBnB Listings for Los Angeles Area

In [3]:
## load pandas and import dataset from GitHub
import pandas as pd

df = pd.read_csv("https://media.githubusercontent.com/media/boneeyah/DS7331_Group/main/Data_Files/airbnb_los_angeles.csv")

In [5]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42041 entries, 0 to 42040
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            42041 non-null  int64  
 1   listing_url                                   42041 non-null  object 
 2   scrape_id                                     42041 non-null  int64  
 3   last_scraped                                  42041 non-null  object 
 4   name                                          42039 non-null  object 
 5   description                                   41102 non-null  object 
 6   neighborhood_overview                         25110 non-null  object 
 7   picture_url                                   42041 non-null  object 
 8   host_id                                       42041 non-null  int64  
 9   host_url                                      42041 non-null 

In [ ]:
df.in

### Business Understanding

### Data Meaning Type

### Data Quality

### Simple Statistics

### Visualize Attributes

### Explore Joint Attributes

### Explore Attributes and Class

### New Features

### Exceptional Work